In [ ]:
flex_title = "Indonesia Fire Danger Rating System"
flex_subtitle = "Nusantara Earth Observation Network"

In [ ]:
import ee
import os
from datetime import date, timedelta

import ipyleaflet
import ipyscales

import ipywidgets as widgets
from IPython.display import display

ee_token = os.environ.get("EARTHENGINE_TOKEN")
if ee_token is not None:
    credential_file_path = os.path.expanduser("~/.config/earthengine/")
    if not os.path.exists(credential_file_path):
        credential = '{"refresh_token":"%s"}' % ee_token
        os.makedirs(credential_file_path, exist_ok=True)
        with open(credential_file_path + "credentials", "w") as file:
            file.write(credential)
ee.Initialize()

In [ ]:
palette = ['blue', 'green', 'yellow', 'red']

component_dict = {
    'Fire Weather Index' : {
        'prefix' : 'FWI',
        'ranges' : [0, 1, 6, 13],
    },
    'Initial Spread Index' : {
        'prefix' : 'ISI',
        'ranges' : [0, 2, 4, 5],
    },
    'Buildup Index' : {
        'prefix' : 'BUI',
        'ranges' : [0, 7, 20, 33],
    },
    'Drought Code' : {
        'prefix' : 'DC',
        'ranges' : [0, 140, 260, 350],
    },
    'Duff Moisture Code' : {
        'prefix' : 'DMC',
        'ranges' : [0, 5, 15, 29],
    },
    'Fine Fuel Moisture Code' : {
        'prefix' : 'FFMC',
        'ranges' : [0, 73, 78, 82],
    },
}

province_dict = {
    'Sumatera Selatan' : {
        'prefix' : 'ID-SS',
    },
    'Jambi' : {
        'prefix' : 'ID-JA',
    },
    'Riau' : {
        'prefix' : 'ID-RI',
    },

}

## Sidebar

### Parameters

In [ ]:
# Province selection
prov_label = widgets.Label(value='Province:')
prov_var = widgets.Dropdown(
    options=[
        'Sumatera Selatan', 'Jambi', 'Riau',
    ],
    value='Sumatera Selatan',
)

# Observation date selection
obs_label = widgets.Label(value='Date:')
obs_var = widgets.DatePicker(
    value=date.today() - timedelta(days=1)
)

# Component selection
comp_label = widgets.Label(value='Component:')
comp_var = widgets.Dropdown(
    options=[
        'Fine Fuel Moisture Code', 'Duff Moisture Code', 
        'Drought Code', 'Initial Spread Index', 
        'Buildup Index', 'Fire Weather Index'
    ],
    value='Fire Weather Index',
)

# Create and display the sidebar
inputs = widgets.VBox([
    prov_label, prov_var, 
    obs_label, obs_var, 
    comp_label, comp_var
])
display(inputs)

### About

This application shows calibrated Canadian Fire Weather Index (FWI) calculated by using data from GSMaP (JAXA) and GFS (NOAA).

For more information, contact:
[Josef Matondang](mailto:admin@josefmtd.com)

## Column

### Fire Weather Index

In [ ]:
# Generate the basemap
Map = ipyleaflet.Map(center=(0,120), zoom=7,
    min_zoom=5, max_zoom=12, scroll_wheel_zoom=True,
    basemap=ipyleaflet.basemaps.CartoDB.Positron,
)

def create_ee_tile_layer(image, vis_params, name):
    # Use the Google Earth Engine eeImage dynamic tiling service
    map_id_dict = image.getMapId(vis_params)
    return ipyleaflet.TileLayer(
        url=map_id_dict['tile_fetcher'].url_format,
        attribution='National Oceanic and Atmospheric Agency, ' + \
            'Japan Aerospace Exploration Agency, Google Earth Engine',
        name=name, opacity=0.6, shown=True
    )

def recenter_map(change):
    # Get value change from callback function
    prov = prov_var.value

    # Add province parameters
    province = ee.FeatureCollection('FAO/GAUL/2015/level1') \
        .filter(ee.Filter.eq('ADM1_NAME', prov))

    # Set map center
    center = ee.Feature(province.first()).centroid() \
        .geometry().getInfo()['coordinates']
    Map.center = center[::-1]

def generate_legend(change):
    # Get value change from callback function
    comp = comp_var.value

    # Create colormap
    ranges = component_dict[comp]['ranges']
    cm = ipyscales.LinearColorScale(
        range=tuple(palette),
        domain=tuple(ranges)
    )

    # Generate legend widget on ipyleaflet
    colorbar = ipyscales.ColorBar(
        colormap=cm, length=250, width=10, orientation='horizontal',
    )
    legend = ipyleaflet.WidgetControl(
        widget=colorbar, position='topright'
    )

    # Update the legend only if there's change
    if change is not None:
        Map.controls = Map.controls[:2]
    Map.add_control(legend)

def generate_fwi_layer(change):
    # Get value change from callback function
    obs = obs_var.value
    comp = comp_var.value
    prov = prov_var.value

    # Obtain ee.Image from Google Cloud Storage
    prefix = province_dict[prov]['prefix']
    url = f'gs://fwi-bicubic-outputs/{prefix}_{obs.isoformat()}_FWI.tif'
    fwi = ee.Image.loadGeoTIFF(url)

    # Visualization parameters
    ranges = component_dict[comp]['ranges']
    vis_params = {
        'min': ranges[0], 
        'max': ranges[-1], 
        'palette': palette,
    }

    # Add overlay layer
    band = component_dict[comp]['prefix']
    layer = create_ee_tile_layer(fwi.select(band), vis_params, comp)

    if change is not None:
        Map.layers = Map.layers[:1]
    Map.add_layer(layer)

In [ ]:
# Update the layer
prov_var.observe(generate_fwi_layer, names="value")
obs_var.observe(generate_fwi_layer, names="value")
comp_var.observe(generate_fwi_layer, names="value")

# Update legend and center
comp_var.observe(generate_legend, names="value")
prov_var.observe(recenter_map, names="value")

# Shows the figure
generate_fwi_layer(None)
generate_legend(None)
recenter_map(None)
display(Map)